# Guide for this Tutorial 

#### these are the main steps for this tutorial


# Dataset Preparation

In [ ]:
import os, zipfile

train_path = '/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_train_20170904.zip'
validation_path = '/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_validation_20170908.zip'
test_a_path = '/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_test_a_20170922.zip'
extract_path = '/media/bitlord/document/ai_challenger/scene_cal/data'

def unzip(zipfile_path, extract_path, zipfile_name):
    zipfile = zipfile.ZipFile(zipfile_path, 'r')
    print('Extracting {} ...'.format(zipfile_name))
    zipfile.extractall()
    zipfile.close()
    print('{} has been extracted.'.format(zipfle_name))

if os.path.exists(extract_path):
    print('Found extraced dataset')
else:
    unzip(train_path, extract_path, 'training dataset')
    unzip(validation_path, extract_path, 'validation dataset')
    unzip(test_a_path, extract_path, 'test dataset')

# Data Presentation

In [1]:
import json
import glob
from scipy.misc import imread, imresize, imsave
import numpy as np

train_features_path = r'/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_train_20170904/scene_train_images_20170904'
train_labels_path = r'/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_train_20170904/scene_train_annotations_20170904.json'
validation_features_path =r'/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_validation_20170908/scene_validation_images_20170908'
validation_labels_path =r'/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_validation_20170908/scene_validation_annotations_20170908.json'
test_a_features_path = r'/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_test_a_20170922/scene_test_a_images_20170922'

# Show train label list
with open(train_labels_path, 'r') as f:
    train_label_list = json.load(f)
    print(train_label_list[:10])
    train_dict = {}
    for image in train_label_list:
        train_dict[image['image_id']] = int(image['label_id'])
    print('\n')
    print(len(train_dict))     

# Show train features list resulting out of memory...

[{u'image_id': u'79f993ae0858ae238b22968c5934d1ddba585ae4.jpg', u'label_id': u'66', u'image_url': u'https://n1-q.mafengwo.net/s1/M00/6B/72/wKgBm04Wc5WzFXU0AAHf09bdpiY84.jpeg?imageView2%2F2%2Fw%2F600%2Fq%2F90'}, {u'image_id': u'e963208fe9e90df0c385f7367bcdb6d0d5d0b165.jpg', u'label_id': u'61', u'image_url': u'http://news.sogou.com/'}, {u'image_id': u'02df5ecbf7c749ccc9d833f129bbd5d9837940ce.jpg', u'label_id': u'64', u'image_url': u'http://img2.fawan.com/2016/12/30/e967f93e7713c57cd2b00b832dd6091a_500x-_90.jpg'}, {u'image_id': u'5620eb385b7567fb087813cf5233b5ceecdeeca3.jpg', u'label_id': u'31', u'image_url': u'https://b1-q.mafengwo.net/s1/M00/F2/C9/wKgBm04Wx3a-gk2FAAKbPKX7E9w91.jpeg?imageView2%2F2%2Fw%2F600%2Fq%2F90'}, {u'image_id': u'f8b4d42001a562fc63b9b39c02531661c0e236ca.jpg', u'label_id': u'19', u'image_url': u'http://news.sogou.com/'}, {u'image_id': u'57e7eb438670a4519041dab1482f2594a92f8a09.jpg', u'label_id': u'11', u'image_url': u'http://www.user2.jqw.com/2014/01/06/1347666/produ

# Initialization features(input) and labels(output) 

In [2]:
import json
from scipy.misc import imread, imresize
import numpy as np
import os

class initialize(object):
    # Get image-label list for train and validation
    def __init__(self, feature_path, label_path):
        self.image_label_dict = {}
        with open(label_path, 'r') as f:
            label_list = json.load(f)
        for image in label_list:
            self.image_label_dict[image['image_id']] = int(image['label_id'])
        self.start = 0
        self.end = 0
        self.length = len(self.image_label_dict) # number of feature images
        self.image_name = list(self.image_label_dict.keys())
        self.feature_path = feature_path
    
    # Read image in feature path, resize and normalize to [-1, 1]
    def get_image(self, image_path, image_size):
        image = imread(image_path)
        image = imresize(image, [image_size, image_size])       
        image = np.array(image).astype(np.float32)
        image = 2 * (image - np.min(image)) / np.ptp(image) - 1
        return image
    
    # Get feature and label batch
    def get_batch(self, batch_size, image_size):
        self.start = self.end
        if self.start >= self.length:
            self.start = 0
        batch_feature = []
        batch_label = []
        index = self.start
        while len(batch_feature) < batch_size:
            if index >= self.length:
                index = 0
            i_image_path = os.path.join(self.feature_path, self.image_name[index])
            i_image = self.get_image(i_image_path, image_size)
            i_label = self.image_label_dict[self.image_name[index]]
            batch_feature.append(i_image)
            batch_label.append(i_label)
            index += 1
        self.end = index
        return batch_feature, batch_label

# Conv Network (CNN) return training accuracy and loss

In [3]:
import tensorflow as tf

def conv_layer(input_layer, filters, strides, is_training):
    layer = tf.layers.conv2d(
        inputs=input_layer, 
        filters=filters, 
        kernel_size=3,
        strides=strides, 
        padding='same', 
        activation=None,
        kernel_initializer=tf.truncated_normal_initializer()
    )
    
    layer = tf.layers.batch_normalization(
        inputs=layer, 
        axis=-1,
        momentum=0.9,
        epsilon=0.001,
        center=True,
        scale=True,
        training=is_training
    )
    
    layer = tf.nn.relu(layer)
    return layer

def fully_connected(input_layer, num_units, is_training):
    layer = tf.layers.dense(input_layer, num_units, use_bias=False, activation=None)
    layer = tf.layers.batch_normalization(layer, training=is_training)
    layer = tf.nn.relu(layer)
    return layer

def conv_network(feature, label, num_class, image_size, keep_prob, is_training):
    input_layer = tf.reshape(feature, [-1, image_size, image_size, 3])
    
    # 16 conv layers with 64, 128, 256, 512 filters.
    layer = conv_layer(input_layer, 64, 1, is_training)
    layer = conv_layer(layer, 64, 1, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 128, 1, is_training)
    layer = conv_layer(layer, 128, 1, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 256, 2, is_training)
    layer = conv_layer(layer, 256, 2, is_training)
    layer = conv_layer(layer, 256, 2, is_training)
    layer = conv_layer(layer, 256, 2, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = conv_layer(layer, 512, 2, is_training)
    layer = tf.layers.max_pooling2d(layer, pool_size=[2, 2], strides=2, padding='same')
    
    shape = layer.get_shape().as_list()
    layer = tf.reshape(layer, shape=[-1, shape[1]*shape[2]*shape[3]])
    layer = fully_connected(layer, 800, is_training)
    layer = tf.nn.dropout(layer, keep_prob)
    logits = tf.layers.dense(layer, 80)
    output = tf.sigmoid(logits)
    
    # Loss and optimizer
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=label))
    
    return logits, loss

# Training network

In [ ]:
def train(train_feature_path, train_label_path, checkpoint_path, num_class, batch_size, image_size, max_step):
    
    train = initialize(train_feature_path, train_label_path)
        
    feature = tf.placeholder(tf.float32, shape=[None, image_size, image_size, 3], name='feature')
    label = tf.placeholder(tf.float32, shape=[None], name='label')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    one_hot_label = tf.one_hot(indices=tf.cast(label, tf.int32), depth=80)
    is_training = tf.placeholder(tf.bool, name='is_training')
    
    logits, loss = conv_network(feature, one_hot_label, num_class, image_size, keep_prob, is_training)
    
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        train_opt = tf.train.AdamOptimizer(learning_rate).minimize(loss)
        
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_label, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
       
    with tf.Session() as sess:
        saver = tf.train.Saver()
        ckpt = tf.train.get_checkpoint_state(checkpoint_path)
        
        if ckpt and ckpt.model_checkpoint_path:
            print('Restore the model from checkpoint {}.'.format(ckpt.model_checkpoint_path))
            start_step = int(ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1])
        else:
            sess.run(tf.global_variables_initializer())
            start_step = 0
            print('Start training from new start.')
        
        for steps in range(start_step, start_step + max_step):
            train_feature_batch, train_label_batch = train.get_batch(batch_size, image_size)
            #print(is_training, feature, label)
            sess.run(train_opt, feed_dict={feature: train_feature_batch, label: train_label_batch, keep_prob: 0.5, is_training: True})
                
            if steps % 10 == 0:
                train_accuracy = sess.run(accuracy, feed_dict={feature: train_feature_batch, label: train_label_batch, keep_prob: 0.5, is_training: False})
                train_loss = sess.run(loss, feed_dict={feature: train_feature_batch, label: train_label_batch, keep_prob: 0.5, is_training: False})
                print('Step {}'.format(steps),
                      'Training Accuracy {:.3f}...'.format(train_accuracy),
                      'Training Loss {:.3f}...'.format(train_loss),
                     ) 
            if steps % 1000 == 0:
                saver.save(sess, checkfile, global_step=steps)
                print('Writing checkpoint at step {}'.format(steps))

        print('Training completed.')

# image pathes
train_feature_path = r'/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_train_20170904/scene_train_images_20170904'
train_label_path = r'/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_train_20170904/scene_train_annotations_20170904.json'
checkpoint_path = './checkpoint/'
checkfile = './checkpoint/model.ckpt'

num_class = 80
batch_size = 64
image_size = 64
max_step = 60000
learning_rate =0.002

train(train_feature_path, train_label_path, checkpoint_path, num_class, batch_size, image_size, max_step)

In [8]:
def test(val_feature_path, val_label_path, checkpoint_dir, num_class, image_size):
    test = initialize(val_feature_path, val_label_path)
    test_images = os.listdir(val_feature_path)
    
    feature = tf.placeholder(tf.float32, shape=[None, image_size, image_size, 3], name='feature')
    label = tf.placeholder(tf.float32, shape=[None], name='label')
    one_hot_label = tf.one_hot(indices=tf.cast(label, tf.int32), depth=80)
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    is_training = tf.placeholder(tf.bool, name='is_training')
    
    logits, loss = conv_network(feature, label, num_class, image_size, keep_prob, is_training )
    values, indices = tf.nn.top_k(logits, 3)

    with tf.Session() as sess:
        saver = tf.train.Saver()
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            print("Restore the model from checkpoint %s" % ckpt.model_checkpoint_path)
            saver.restore(sess, ckpt.model_checkpoint_path)
            start_step = int(ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1])
        else:
            raise Exception('No checkpoint found')
        
        result = []
        for test_image in test_images:
            temp_dict = {}
            x = test.get_image(os.path.join(val_feature_path, test_image), image_size)
            predictions = np.squeeze(sess.run(indices, feed_dict={feature: np.expand_dims(x, axis=0), keep_prob: 1}), axis=0)
            temp_dict['image_id'] = test_image
            temp_dict['label_id'] = predictions.tolist()
            result.append(temp_dict)
            print('image %s is %d, %d, %d' % (test_image, predictions[0], predictions[1], predictions[2]))
            
        with open('submit.json', 'w') as f:
            json.dump(result, f)
            print('Write result json file, num is %d' % len(result))

In [9]:
val_feature_path = r'/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_test_a_20170922/scene_test_a_images_20170922'
val_label_path = r'/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_validation_20170908/scene_validation_annotations_20170908.json'
checkpoint_dir = r'./checkpoint/'
num_class = 80
image_size = 64

test(val_feature_path, val_label_path, checkpoint_dir, num_class, image_size)

Restore the model from checkpoint ./checkpoint/model.ckpt-46000
INFO:tensorflow:Restoring parameters from ./checkpoint/model.ckpt-46000


InvalidArgumentError: You must feed a value for placeholder tensor 'is_training' with dtype bool
	 [[Node: is_training = Placeholder[dtype=DT_BOOL, shape=<unknown>, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'is_training', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "/usr/local/lib/python2.7/dis

In [ ]:
train_feature_path = r'/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_train_20170904/scene_train_images_20170904'
train_label_path = r'/media/bitlord/document/ai_challenger/scene_cal/data/ai_challenger_scene_train_20170904/scene_train_annotations_20170904.json'
checkpoint_path = './checkpoint/'
checkfile = './checkpoint/model.ckpt'